In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json

In [6]:
class SeparableConv2D(nn.Module):
    '''Separable convolution'''
    def __init__(self, in_channels, out_channels, stride=1):
        super(SeparableConv2D, self).__init__()
        self.dw_conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=False),
        )
        self.pw_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=False),
        )

    def forward(self, x):
        x = self.dw_conv(x)
        x = self.pw_conv(x)
        return x

CIRCOM_PRIME = 21888242871839275222246405745257275088548364400416034343698204186575808495617
MAX_POSITIVE = CIRCOM_PRIME // 2
MAX_NEGATIVE = MAX_POSITIVE + 1 # The most positive number
CIRCOM_NEGATIVE_1 = 21888242871839275222246405745257275088548364400416034343698204186575808495617 - 1

def from_circom(x):
    if type(x) != int:
        x = int(x)
    if x > MAX_POSITIVE: 
        return x - CIRCOM_PRIME
    return x
    
def to_circom(x):
    if type(x) != int:
        x = int(x)
    if x < 0:
        return x + CIRCOM_PRIME 
    return x

In [79]:
input = torch.randn((1, 3, 5, 5))

model = SeparableConv2D(3, 3)
expected = model(input)

In [80]:
def Conv2DInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[[str(weights[i][j][k][l] % p) for l in range(nFilters)] for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(bias[i] % p) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    remainder = [[[None for _ in range(nFilters)] for _ in range((nCols - kernelSize)//strides + 1)] for _ in range((nRows - kernelSize)//strides + 1)]
    for i in range((nRows - kernelSize)//strides + 1):
        for j in range((nCols - kernelSize)//strides + 1):
            for m in range(nFilters):
                for k in range(nChannels):
                    for x in range(kernelSize):
                        for y in range(kernelSize):
                            out[i][j][m] += input[i*strides+x][j*strides+y][k] * weights[x][y][k][m]
                out[i][j][m] += bias[m]
                remainder[i][j][m] = str(out[i][j][m] % n)
                out[i][j][m] = str(out[i][j][m] // n % p)
    return Input, Weights, Bias, out, remainder
    
def DepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    out = np.zeros((outRows, outCols, nFilters))
    remainder = np.zeros((outRows, outCols, nFilters))
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row, col, channel] += input[row*strides+x, col*strides+y, channel] * weights[x, y, channel]
            # remainder[i][j][m] = str(out[i][j][m] % n)
                
                out[row][col][channel] += bias[channel]
                remainder[row][col][channel] = out[row][col][channel] % n
                out[row][col][channel] = out[row][col][channel] / n
                            
    return out, remainder

weights = model.dw_conv[0].weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()
# input = torch.randn((1, 8, 32, 32))

expected = model.dw_conv[0](input).detach().numpy()

# # Converting to H x W x C
padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

actual, rem = DepthwiseConv(7, 7, 3, 3, 3, 1, 1, padded, weights, bias)
expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual, atol=0.00001))

In [82]:
EXPONENT = 8
weights = model.dw_conv[0].weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()
# input = torch.randn((1, 8, 32, 32))

expected = model.dw_conv[0](input).detach().numpy()

# # Converting to H x W x C
padded = F.pad(input, (1,1,1,1), "constant", 0)
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

quantized_image = padded * 10**EXPONENT
quantized_weights = weights * 10**EXPONENT

actual, _ = DepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image, quantized_weights, bias)
actual = actual / 10**(EXPONENT)

expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual, atol=0.00001))

In [92]:
print(quantized_image.shape)
input_json_path = "depthwiseConv2D_input.json"
print(quantized_image.shape)
with open(input_json_path, "w") as input_file:
    json.dump({"in": quantized_image.round().astype(int).tolist(),
               "weights": quantized_weights.round().tolist(),
               "bias": bias.astype(int).tolist(),
              },
              input_file)


(7, 7, 3)
(7, 7, 3)
